# LLM-Based Energy Data Analysis Notebook
### Author: Shivam Pal
### Date: 01/06/2025
### Description: Energy consumption analysis using LLM-generated code

In [148]:
import pandas as pd
import requests
import json

**Mount Drive**

In [14]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**Set Environment Varaible**

In [157]:
GROQ_API_KEY = "Your GROQ API KEY" #@param {type:"string"}
model = "meta-llama/llama-4-scout-17b-16e-instruct" #@param ["meta-llama/llama-4-scout-17b-16e-instruct", "deepseek-r1-distill-llama-70b","gemma2-9b-it", "compound-beta" , "meta-llama/llama-4-maverick-17b-128e-instruct"]
Dataset_FilePath = "FilePath" #@param {type:"string"}

**Load Dataset**

In [152]:
df = pd.read_csv(Dataset_FilePath,
             	sep=';',
             	parse_dates={'datetime': ['Date', 'Time']},
             	infer_datetime_format=True,
             	na_values=['?'],
             	low_memory=False)

df = df.dropna()
df['Global_active_power'] = df['Global_active_power'].astype(float)
df = df.set_index('datetime')

**Class for LLM Inference**

In [153]:
class LLMInterface:
    """LLM interface for code generation"""

    def __init__(self, api_key: str):
        """Initialize with Groq API key"""
        self.api_key = api_key
        self.base_url = "https://api.groq.com/openai/v1/chat/completions"

    def generate_code(self, query: str, model: str = 'meta-llama/llama-4-scout-17b-16e-instruct', context: str = "") -> str:
        """Generate pandas code from natural language query"""

        system_prompt = f"""You are an expert data scientist specializing in energy data analysis.
        You will be given a natural language query about a pandas DataFrame named `df`.
        Generate clean, efficient pandas (and matplotlib or seaborn if plotting is required) code to answer the query.
        we have alrady loaded df and you have to only provide code to get output of query.

        Dataset Metadata:
        - Name: df
        - Index: DatetimeIndex
        - Range: 2006-12-16 17:24:00 to 2010-11-26 21:02:00
        - Frequency: 1 min Interval
        - Data columns (total 7 columns):
          #   Column                 Dtype       Description
          ---  ------                 -----
          0   Global_active_power    float64     Household active power consumption (in kilowatt)
          1   Global_reactive_power  float64     Reactive power (in kilowatt)
          2   Voltage                float64     Average voltage (in volts)
          3   Global_intensity       float64     Average current (in amperes)
          4   Sub_metering_1         float64     Energy used in the kitchen (in watt-hours)
          5   Sub_metering_2         float64     Energy used in the laundry room (in watt-hours)
          6   Sub_metering_3         float64     Energy used for water heater and air conditioning (in watt-hours)
        - Total Rows: 2049280 entries

        Instructions:
        - Do not modify, add, or delete any rows or columns in the original DataFrame 'df'.
        - Parse the question and write precise pandas code that returns the final answer.
        - Output should be minimal: only the final numerical/statistical result, table, or plot.
        - Code must be syntactically correct and logically valid.
        - Use datetime indexing, filtering, grouping, resampling, or aggregation where appropriate.
        - Comment the code for clarity if multi-step logic is required.
        - Output should be in a single Python code block.

        {context}
        """

        headers = {
            "Authorization": f"Bearer {self.api_key}",
            "Content-Type": "application/json"
        }

        payload = {
            "model": model,
            "messages": [
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": f"Query: {query}"}
            ],
            "temperature": 0.1,
            "max_tokens": 1000
        }

        try:
            response = requests.post(self.base_url, json=payload, headers=headers)
            response.raise_for_status()

            result = response.json()
            code = result['choices'][0]['message']['content']

            # Clean the code (remove markdown formatting if present)
            if '```python' in code:
                code = code.split('```python')[1].split('```')[0]
            elif '```' in code:
                code = code.split('```')[1].split('```')[0]

            return code.strip()

        except Exception as e:
            print(f"Error generating code: {str(e)}")
            return f"Error: Could not generate code for query: {query}"

llm = LLMInterface(GROQ_API_KEY)

Function to Execute LLM Generated Code

In [154]:
def execute_code(query):
  """Execute LLM Generated Code"""

  generated_code = llm.generate_code(query, model=model)

  print(f"Generated Code:\n{generated_code}\n\nOutput:\n")

  try:
    exec(generated_code)
  except Exception as e:
    print(f"Error executing code: {str(e)}")

1. What was the average active power consumption in March 2007?
2. What hour of the day had the highest power usage on Christmas 2006?
3. Compare energy usage (Global_active_power) on weekdays vs weekends.
4. Find days where energy consumption exceeded 5 kWh.
5. Plot the energy usage trend for the first week of January 2007.
6. Find the average voltage for each day of the first week of February 2007.
7. What is the correlation between global active power and sub-metering values?

In [156]:
query="Find the average voltage for each day of the first week of February 2007."  #@param {type: "string"}
execute_code(query)

Generated Code:
# Filter data for February 2007
feb_2007 = df[(df.index >= '2007-02-01') & (df.index < '2007-03-01')]

# Filter data for the first week of February 2007
first_week_feb_2007 = feb_2007[(feb_2007.index.dayofweek < 7) & (feb_2007.index.day <= 7)]

# Group by day and calculate average voltage
avg_voltage_per_day = first_week_feb_2007['Voltage'].resample('D').mean()

print(avg_voltage_per_day)

Output:

datetime
2007-02-01    240.392139
2007-02-02    240.334389
2007-02-03    238.910194
2007-02-04    239.633333
2007-02-05    240.810312
2007-02-06    241.264160
2007-02-07    241.149694
Freq: D, Name: Voltage, dtype: float64


# Thank You